# Parte 1: Utilidade

## Importar os pacotes necessários

In [4]:
# importar os pacotes necessários para fazer análise
import pandas as pd, numpy as np
import os, re
import requests
import zipfile

## Definir funções para processar os dados

In [ ]:
# definir função que baixa os dados
def baixar_tse():
    URL = 'http://agencia.tse.jus.br/estatistica/sead/odsele/motivo_cassacao/motivo_cassacao_2016.zip'
    r = requests.get(URL)
    with open('../dados/motivo_cassacao2016.zip', 'wb') as arquivo:
        arquivo.write(r.content)
    return 'Motivo(s) para Indeferimento 2016 baixado(s) com sucesso.'

# definir função de descomprime arquivo do tse
def unzip_tse():
    with zipfile.ZipFile('../dados/motivo_cassacao2016.zip', 'r') as zip:
        zip.extractall('../dados')
    return 'Arquivo(s) descomprimido(s) com sucesso.'

# definir função para juntar os dados do tse
def juntar_tse():
    regex = re.compile(r'motivo.*csv$')
    juntar_tse.kwargs = {
        'engine': 'python', 'encoding': 'latin1', 'sep':';', 'quoting': 1,
        'dtype': str
    }
    arquivos = os.listdir('../dados')
    arquivos = list(filter(regex.search, arquivos))
    arquivos = [
        os.path.join(os.path.realpath('../dados'), arquivo) \
        for arquivo in arquivos
    ]
    dados = pd.concat(
        [pd.read_csv(arquivo, **juntar_tse.kwargs) for arquivo in arquivos],
        ignore_index=True
    )
    dados = dados[dados['DS_MOTIVO_CASSACAO'].notna()]
    print('Arquivo(s) juntado(s) com sucesso.')
    return dados

## Executar funções

In [ ]:
# executar função
baixar_tse()

In [ ]:
# executar função
unzip_tse()

In [ ]:
# executar função e salvar o resultado no banco de dados chamado "candidaturas"
candidaturas = juntar_tse()

## Exibir dados preliminares

In [ ]:
# mostrar os dados
candidaturas.sample(10)

In [ ]:
# verificar os motivos para cassação
list(candidaturas['DS_MOTIVO_CASSACAO'].unique())

## Focar no Paraná

In [ ]:
# focar em 10 candidatos no paraná
amostra_pr = candidaturas[candidaturas['SG_UF'] == 'PR'].sample(10, random_state=67)

In [ ]:
# mostrar quais informações nós obtivemos
amostra_pr.columns.to_list()

In [ ]:
# juntar os motivos de cassação com outros dados dos candidatos?
candidatos_pr = pd.read_csv('../dados/consulta_cand_2016_PR.csv', **juntar_tse.kwargs)

In [ ]:
# puxar os números do protocolo da candidatura
amostra_pr = amostra_pr.merge(candidatos_pr, on='SQ_CANDIDATO')

In [ ]:
# disponibilizar na tela os números dos protocolos de candidatura
# amostra_pr.columns
amostra_pr[['NM_CANDIDATO', 'NM_UE_x', 'DS_CARGO', 'NR_PROTOCOLO_CANDIDATURA']]

In [ ]:
# criar link para raspar documentos
links = [
    f'http://inter03.tse.jus.br/sadpPush/ExibirDadosProcesso.do?' \
    f'nprot={nprot}&comboTribunal=pr' \
    for nprot in amostra_pr['NR_PROTOCOLO_CANDIDATURA'].to_list()
]

# disponibilizar os links para a gente
for link in links: 
    print(link)

## Baixar e processar os dados

In [ ]:
# # baixar os arquivos
# raspador = tse.scraper(navegador_automatico.browser)

# # baixar todos
# for i, url in enumerate(documentos_link):
#     raspador.decision(url=url, filename=f'decisão_{i+1}')

# # fechar
# navegador_automatico.browser.quit()

In [16]:
# importar meu pacote de análise
from tse import parser

In [18]:
# construir path para os arquivos
decisões = [f'../dados/decisão_{i}.html' for i in range(0, 10)]

In [19]:
# definir função para transformar os dados do sumário em tabela
def tabelar_sumário(decisão, transpor=True):
    sumário = pd.DataFrame(parser(decisão).parse_summary())
    if transpor: 
        sumário = sumário.T
    return sumário

In [20]:
# produzir tabela
tabelar_sumário(decisões[1])

,0
case,Nº 0000299-72.2016.6.16.0127 - REGISTRO DE CAN...
town,NOVA OLÍMPIA-PRN.° Origem:
prot,1225872016 - 15/08/2016 18:13
claimants,COLIGAÇÃO GOVERNANDO COM O POVO (PMDB / PSC / ...
defendant,"VERA ILSA BALDUINO DA SILVA, CARGO VEREADOR, N..."
judge,FERNANDA BATISTA DORNELLES
subject,DIREITO ELEITORAL - Eleições - Candidatos - Re...
district,127ZE-127 ZONA ELEITORAL
stage,09/02/2017 12:21-Arquivado na seção


In [21]:
# criar banco de dados
sumários = pd.concat([tabelar_sumário(decisão, False) for decisão in decisões], ignore_index=True)
sumários

,case,town,prot,claimants,defendant,judge,subject,district,stage
0,Nº 0000304-61.2016.6.16.0041 - REGISTRO DE CAN...,LONDRINA-PRN.° Origem:,1194392016 - 15/08/2016 17:09,COLIGAÇÃO LONDRINA PRA FRENTE (PP / PTB),"MAURICIO QUIMENTON COSTA, CARGO VEREADOR, Nº: ...",MATHEUS ORLANDI MENDES,DIREITO ELEITORAL - Eleições - Candidatos - Re...,041ZE-041 ZONA ELEITORAL,25/09/2016 15:56-Arquivado na seção
1,Nº 0000299-72.2016.6.16.0127 - REGISTRO DE CAN...,NOVA OLÍMPIA-PRN.° Origem:,1225872016 - 15/08/2016 18:13,COLIGAÇÃO GOVERNANDO COM O POVO (PMDB / PSC / ...,"VERA ILSA BALDUINO DA SILVA, CARGO VEREADOR, N...",FERNANDA BATISTA DORNELLES,DIREITO ELEITORAL - Eleições - Candidatos - Re...,127ZE-127 ZONA ELEITORAL,09/02/2017 12:21-Arquivado na seção
2,Nº 0000050-36.2016.6.16.0123 - REGISTRO DE CAN...,SÃO JORGE DO PATROCÍNIO-PRN.° Origem:,1013362016 - 12/08/2016 18:24,"COLIGAÇÃO PSD, PP, PTN (PSD / PP / PTN)","FERNANDO DANIEL SCHMITD, CARGO VEREADOR, Nº: 5...",GUILHERME ARANDA CASTRO DOS SANTOS,DIREITO ELEITORAL - Eleições - Candidatos - Re...,123ZE-123 ZONA ELEITORAL,23/11/2016 12:10-Arquivado na seção
3,Nº 0000161-14.2016.6.16.0028 - REGISTRO DE CAN...,APUCARANA-PRN.° Origem:,1134682016 - 15/08/2016 14:58,"COLIGAÇÃO PTB-PMDB-PPS (PTB / PMDB / PPS), COL...","JUÍZO ELEITORAL DA 28ª ZONA, TRE-PR",RENATA BOLZAN JAURIS,DIREITO ELEITORAL - Eleições - Candidatos - Re...,028ZE-028 ZONA ELEITORAL,20/03/2017 16:41-Arquivado na seção
4,Nº 0000186-20.2016.6.16.0095 - REGISTRO DE CAN...,COLORADO-PRN.° Origem:,1065522016 - 15/08/2016 12:32,COLIGAÇÃO JUNTOS POR COLORADO (PPL / PT DO B /...,"LEANDRO PIRES DA SILVA, CARGO VEREADOR, Nº: 54000",LUCIANA PAULA KULEVICZ,DIREITO ELEITORAL - Eleições - Candidatos - Re...,095ZE-095 ZONA ELEITORAL,07/03/2017 17:30-Arquivado na seção
5,Nº 0000280-70.2016.6.16.0061 - REGISTRO DE CAN...,ARAPONGAS-PRN.° Origem:,1182182016 - 15/08/2016 16:37,COLIGAÇÃO PP E SD (PP / SD),"JOSE MARIA DA SILVA, CARGO VEREADOR, Nº: 11611",RENATA MARIA FERNANDES SASSI FANTIN,DIREITO ELEITORAL - Eleições - Candidatos - Re...,061ZE-061 ZONA ELEITORAL,14/03/2017 17:44-Arquivado na seção
6,Nº 0000180-76.2016.6.16.0074 - REGISTRO DE CAN...,PEABIRU-PRN.° Origem:,1247862016 - 15/08/2016 18:54,COLIGAÇÃO PSB - PDT - PEN - PT DO B - PSC - PR...,"JOSINEL PEZINI DOS SANTOS, CARGO VEREADOR, Nº:...",PAULO EDUARDO MARQUES PEQUITO,DIREITO ELEITORAL - Eleições - Candidatos - Re...,074ZE-074 ZONA ELEITORAL,19/05/2020 18:50-Certidão
7,Nº 0000259-85.2016.6.16.0161 - REGISTRO DE CAN...,GUARATUBA-PRN.° Origem:,1969862016 - 12/09/2016 17:54,COLIGAÇÃO GUARATUBA MAIS HUMANA (PMDB / PT),"MARIA NEUSA DE LIMA, CARGO VEREADOR, Nº: 13321",GIOVANNA DE SÁ RECHIA,DIREITO ELEITORAL - Eleições - Candidatos - Re...,161ZE-161 ZONA ELEITORAL,13/04/2017 16:35-Arquivado na seção
8,Nº 0000271-55.2016.6.16.0111 - REGISTRO DE CAN...,TELÊMACO BORBA-PRN.° Origem:,1211272016 - 15/08/2016 17:44,COLIGAÇÃO JUNTOS POR TELÊMACO BORBA (PMB / PSD...,"VINICIUS RIBEIRO RAMOS, CARGO VEREADOR, Nº: 45000",MARCELO FURLANETTO DA FONSECA,DIREITO ELEITORAL - Eleições - Candidatos - Re...,111ZE-111 ZONA ELEITORAL,31/03/2017 15:39-Arquivado na seção
9,Nº 0000186-10.2016.6.16.0163 - REGISTRO DE CAN...,QUEDAS DO IGUAÇU-PRN.° Origem:,1151382016 - 15/08/2016 15:27,COLIGAÇÃO A FORÇA QUE VEM DO POVO (PMDB / PDT ...,"PAULA TERESINHA REVERS, CARGO VEREADOR, Nº: 27555",PAULA CHEDID MAGALHÃES,DIREITO ELEITORAL - Eleições - Candidatos - Re...,163ZE-163 ZONA ELEITORAL,14/03/2017 16:46-Arquivado na seção


In [22]:
# onde estão estes processos? nas seguintes zonas eleitorais
sumários['district']

0    041ZE-041 ZONA ELEITORAL
1    127ZE-127 ZONA ELEITORAL
2    123ZE-123 ZONA ELEITORAL
3    028ZE-028 ZONA ELEITORAL
4    095ZE-095 ZONA ELEITORAL
5    061ZE-061 ZONA ELEITORAL
6    074ZE-074 ZONA ELEITORAL
7    161ZE-161 ZONA ELEITORAL
8    111ZE-111 ZONA ELEITORAL
9    163ZE-163 ZONA ELEITORAL
Name: district, dtype: object

## Parte 3: Processamento de Linguagem Natural (NLP)

In [1]:
# importar pacotes
import spacy
from bs4 import BeautifulSoup

# criar ferramentas de análise de texto
nlp_en = spacy.load('en_core_web_sm')
nlp_pt = spacy.load('pt_core_news_sm')

## Exemplo Genérico

In [2]:
# definir função que baixa uma reportagem de jornal
def baixar_reportagem():
    URL = 'https://www.nytimes.com/2020/05/12/well/family/coronavirus-children-covid-19.html'
    r = requests.get(URL)
    soup = BeautifulSoup(r.content)
    text = soup.find('section', attrs={'name': 'articleBody'}).get_text()
    return text

In [5]:
# baixar reportagem do NYTimes
baixar_reportagem()

'As we learn more about children and Covid-19, new research is reshaping some of our thinking. It continues to be true that children, as a group, have been relatively spared, but there is evidence that some may become very sick, and we are beginning to learn more about who may be most at risk, and what parents need to watch for.This past week there were reports of children hospitalized in different locations, including New York City, with a multisystem inflammatory disease that has killed three children. In addition, new research continues to be published describing the ways that the virus can behave in children, which is not always how it behaves in adults.[Read more: The Centers for Disease Control and Prevention’s list of coronavirus symptoms in children]A clinical report published Tuesday in Frontiers in Pediatrics describes five children in China who were admitted to the hospital with nonrespiratory symptoms but who turned out to have Covid, including the characteristic lung abnor

In [6]:
# salvar texto
texto = baixar_reportagem()

In [7]:
# processar o texto
doc_en = nlp_en(texto)

In [8]:
# contar o número de frases
frases = list(doc_en.sents)
len(frases)

61

In [14]:
# mostrar uma frase qualquer
frases[15]

David Baud, the head of the obstetrics service at Lausanne University Hospital in Switzerland, who was the lead author of the letter, said that the most likely way for the virus to reach the placenta is through the bloodstream.

In [10]:
# definir função para investigar o conteúdo de uma frase
def investigar_frase(frase):
    resultado = [{'token': ent.text, 'tipo': ent.label_ , 'descrição': spacy.explain(ent.label_)} for ent in frase.ents]
    return pd.DataFrame(resultado)

In [13]:
# testar reconhecimento do texto
investigar_frase(frases[15])

,token,tipo,descrição
0,David Baud,PERSON,"People, including fictional"
1,Lausanne University Hospital,ORG,"Companies, agencies, institutions, etc."
2,Switzerland,GPE,"Countries, cities, states"


## Exemplo Real

In [81]:
# abrir uma coletânea de decisões judiciais dentre todas
decisão = pd.DataFrame(parser(decisões[2]).parse_details())
decisão

,shead,sbody
0,Despacho em 08/11/2016 - RCAND Nº 5036 DR GUIL...,Autos 50-36.2016.6.16.0123Vistos etc.Tendo em...
1,Sentença em 08/09/2016 - RCAND Nº 5036 DR GUIL...,SENTENÇAProcesso nº: 50-36.2016.6.16.0123 - R...


In [82]:
# extrair texto da decisão
texto = decisão.loc[1, 'sbody']
texto

' SENTENÇAProcesso nº: 50-36.2016.6.16.0123 - REGISTRO DE CANDIDATURA Requerente: FERNANDO DANIEL SCHMIDT1. RelatórioA COLIGAÇÃO PSD/PP/PTN ingressou com requerimento de pedido de candidatura para o cargo de vereador, em favor de FERNANDO DANIEL SCHMIDT.A ilustre representante do Ministério Público Eleitoral, no uso de suas atribuições legais, ajuizou ação de impugnação ao registro de candidatura (fls. 13/15) em desfavor do candidato acima, sustentando, em síntese, sua inelegibilidade, nos termos do art. 1º, inciso I, alínea "e" da Lei Complementar 64/90.Discorre a respeito da declaração de constitucionalidade da Lei Complementar 135/2010, chamada Lei da Ficha Limpa, sobre sua aplicabilidade aos fatos ocorridos antes da sua vigência.Afirma que a condenação criminal do candidato (Ação Penal 2003.0000028-9) à pena de 08 (oito) meses de detenção, como incurso no artigo 163, parágrafo único, inciso III (dano ao patrimônio público) e art. 329 (resistência), ambos do Código Penal, transitada

In [83]:
doc_pt = nlp_pt(texto)

In [84]:
frases = list(doc_pt.sents)

In [85]:
pd.concat([pd.DataFrame(investigar_frase(frase)) for frase in frases])

,token,tipo,descrição
0,SENTENÇAProcesso,MISC,"Miscellaneous entities, e.g. events, nationali..."
1,REGISTRO DE CANDIDATURA,ORG,"Companies, agencies, institutions, etc."
0,FERNANDO DANIEL,MISC,"Miscellaneous entities, e.g. events, nationali..."
0,COLIGAÇÃO PSD,MISC,"Miscellaneous entities, e.g. events, nationali..."
1,PP,ORG,"Companies, agencies, institutions, etc."
...,...,...,...
2,ALTÔNIA,ORG,"Companies, agencies, institutions, etc."
3,PR,ORG,"Companies, agencies, institutions, etc."
0,FERNANDO DANIEL SCHMIDT.Por,MISC,"Miscellaneous entities, e.g. events, nationali..."
0,Altônia,LOC,"Non-GPE locations, mountain ranges, bodies of ..."
